In [64]:
from io import BytesIO
from kfp.components import load_component
from os.path import join as path
from requests import get as GET
import tarfile
from tarfile import TarError
from urllib.parse import urlparse
import zipfile

import kfp

In [65]:
url = 'gs://ml-pipeline-playground/coin.tar.gz'
http = 'https://storage.googleapis.com/ml-pipeline-playground/coin.tar.gz'
dir = '/Users/ryan/c/pipelines/backend'
tar_gz = path(dir, 'coin.tar.gz')
tar = path(dir, 'coin.tar')
zip = path(dir, 'coin.zip')
yaml = path(dir, 'pipeline.yaml')

In [117]:
file = '/Users/ryan/c/pipelines/samples/core/condition/condition.py.zip'

In [66]:
resp = GET(http); resp

<Response [200]>

In [67]:
resp.content

b'\x1f\x8b\x08\x08\xc3\xbc\xe4[\x02\xffcondition.py.tar\x00\xed\x18\xbbn\xdb0\xd0\xb3\xbf\x82\xc8\xe2\x14\xa8d=\xea\x18U\x90\xa1K\x97\x02]\n\xb4C\xd1\x81\xa1\x18\x9b\xb1D\xb2$\x95\xc0M\xf3\xef=K\xb2L\xc5\x92\xe0$\x8e\x9b\x87\xce\x83%\xf2xo\xdd\x83\x92I\x9a0N\xdd%N\x93\xc1\xd3\x80\x07p\xe2y\x03\xdf\xf3\xc2\x89\xe7\x0f\xbc`2\x9d\x04a\xbe\xeey\xfe\xf4\x83\x1f\x0e\xfc0\x9c\xfa~x\x12z\xb0\xeeO\xa6\xd3\xc9\x00y\x83\x03@\xa6\rV\x08\r~\xb3\x94\xf1\xd9e+\x9eT,\xc5j9S"\x93mJ\x02T\xff/\x04\xb0d\xdf\xa9\xd2L\xf0\x08a5\x13R\x89K\x97\x89\xf1\x95\x8f\x139\xc7\xfep\xc1x\x1c\xa1\x1fB-.\x12q=L\xa9\xc1168\x1a"4\xa3\x9c*l\xe8W\x9c\xd2\x08\xc92\x92\x9c\x8b\x84I\x87\x08\xc6\x9d\xa1\x96\x94\xac0\x81r\x96Rn\xf4\xea\x05!\x89\x15\x1c1\xc07B?\x7f\xc1\x12l\xa9\xa5\x84#\xa6\x89\x0e h\xaa\xae\x18\xa1\x9f\x08\x11\x197w\x18\xaa\x8c\x83 \x80eh*\x13\x10(g\xe3 \xc6e\xb6fYcZ\xae8\x88\xe7\x84rF"3\x80\x9d\xef\x14\xabD\xf0\x98\x190\x8c\xe3\xe7\xab\xdaPY\x1eu\xe0wG\xa56\x1e]|\xd6p\x85\x93\x0c\xb6G77\x85\xc4\xee\x86\x8ck\x9d\

In [68]:
resp.ok

True

In [98]:
def url_or_file_to_bytes(url):
  parsed = urlparse(url)
  if not parsed.scheme:
    with open(url, 'rb') as f:
        return f.read()
  else:
    resp = GET(url)
    assert resp.ok
    return resp.content

In [105]:
from kfp import components

In [162]:
swf_yaml_path = '/Users/ryan/c/pipelines/backend/src/crd/samples/scheduledworkflow/kfp.yaml'

In [163]:
import yaml

In [182]:
with open(swf_yaml_path, 'r') as f:
    swf_spec = yaml.safe_load(f)

In [165]:
swf_spec

{'apiVersion': 'kubeflow.org/v1beta1',
 'kind': 'ScheduledWorkflow',
 'metadata': {'name': 'kfp-test-coin-1x'},
 'spec': {'description': 'kfp test (coin.tar.gz, once)',
  'enabled': True,
  'maxHistory': 10,
  'trigger': {'cronSchedule': {'cron': '1 * * * * *'}},
  'workflow': {'parameters': [{'name': 'datetime',
     'value': '[[ScheduledTime.20060102-15:04:05]]'}],
   'spec': {'arguments': {'parameters': [{'name': 'datetime'}]},
    'entrypoint': 'kfp',
    'templates': [{'container': {'args': ['{{inputs.parameters.datetime}}'],
       'command': ['python', 'swf_kfp.py'],
       'image': 'swf-kfp:3'},
      'inputs': {'parameters': [{'name': 'datetime'}]},
      'name': 'kfp'}]}}}}

In [183]:
import json

In [185]:
from pprint import pprint

In [186]:
pprint(swf_spec)

{'apiVersion': 'kubeflow.org/v1beta1',
 'kind': 'ScheduledWorkflow',
 'metadata': {'name': ''},
 'spec': {'description': '',
          'enabled': True,
          'maxHistory': 10,
          'trigger': {'cronSchedule': {'cron': '* * * * *'}},
          'workflow': {'parameters': [{'name': 'datetime',
                                       'value': '[[ScheduledTime.20060102-15:04:05]]'},
                                      {'name': 'pipeline_yaml', 'value': ''}],
                       'spec': {'arguments': {'parameters': [{'name': 'datetime'},
                                                             {'name': 'pipeline_yaml'}]},
                                'entrypoint': 'kfp',
                                'templates': [{'container': {'args': ['--datetime',
                                                                      '{{inputs.parameters.datetime}}',
                                                                      '--pipeline_yaml',
                             

In [192]:
def get_metadata(pipeline):
    if 'metadata' not in pipeline:
        raise Exception('No "metadata" found in pipeline')
    return pipeline['metadata']

In [214]:
def get_annotations(pipeline):
    metadata = get_metadata(pipeline)
    if 'annotations' not in metadata:
        raise Exception('No "annotations" found in pipeline "metadata"')
    annotations = metadata['annotations']
    if 'pipelines.kubeflow.org/pipeline_spec' not in annotations:
        raise Exception('"pipelines.kubeflow.org/pipeline_spec" not found in pipeline metadata.annotations')
    annotations = json.loads(annotations['pipelines.kubeflow.org/pipeline_spec'])
    return annotations['name'], annotations['description']

In [215]:
def get_name(pipeline):
    metadata = get_metadata(pipeline)

    if 'generateName' not in metadata:
        raise Exception('No "generateName" found in pipeline metadata')

    name = metadata['generateName']
    if name[-1] == '-':
        name = name[:-1]

    return name

In [216]:
def get_description(pipeline):
    (_, description) = get_annotations(pipeline)
    return description

In [217]:
def try_extract_pipeline_tar(bytes):
    try:
        with tarfile.open(fileobj=BytesIO(bytes), mode='r') as f:
            names = f.getnames()
            if names == ['pipeline.yaml']:
                tar_info = f.getmember('pipeline.yaml')
                if tar_info.isfile():
                    return f.extractfile(tar_info).read()
                raise Exception('"pipeline.yaml" in TAR archive is not a regular file')
            raise Exception('Expected TAR archive to contain only a "pipeline.yaml"; found %s' % names)
    except TarError:
        return None

In [218]:
def try_extract_pipeline_zip(bytes):
    try:
        with ZipFile(BytesIO(bytes), mode='r') as f:
            names = f.namelist()
            if names == ['pipeline.yaml']:
                return f.read('pipeline.yaml')
            raise Exception('Expected ZIP archive to contain only a "pipeline.yaml"; found %s' % names)
    except BadZipFile:
        return None

In [219]:
condition_zip_path = '/Users/ryan/c/pipelines/samples/core/condition/condition.py.zip'

In [220]:
def load_pipeline_yaml(path):
    bytes = url_or_file_to_bytes(path)
    pipeline_yaml = try_extract_pipeline_tar(bytes)
    if pipeline_yaml is None:
        pipeline_yaml = try_extract_pipeline_zip(bytes)
        if pipeline_yaml is None:
            pipeline_yaml = bytes
    return yaml.safe_load(pipeline_yaml)

In [247]:
DEFAULT_CRON_SCHEDULE = "/10 * * * * *"
def make_swf_kfp(
    pipeline_path, 
    name=None, description=None, 
    cron=None, intervalSecond=None, 
    start=None, end=None,
    maxHistory=10, enabled=True
):
    """"""
    pipeline = load_pipeline_yaml(pipeline_path)
    
    with open(swf_yaml_path, 'r') as f:
        swf = yaml.safe_load(f)
    
    spec = swf['spec']
    
    if name is None:
        name = get_name(pipeline)
    
    if description is None:
        description = get_description(pipeline)
    
    if (cron is not None) and (intervalSecond is not None):
        raise Exception('At most one of {"cron","interval"} should be provided; received cron %s, interval %s' % (cron, intervalSecond))
    
    if (cron is None) and (intervalSecond is None):
        cron = DEFAULT_CRON_SCHEDULE
        
    schedule = {}
    if start is not None:
        schedule['startTime'] = start
    if end is not None:
        schedule['endTime'] = end
    
    msg_parts = []    
    trigger = {}
    if cron is not None:
        schedule['cron'] = cron
        msg_parts.append('cron: %s' % cron)
        trigger['cronSchedule'] = schedule
    else:
        schedule['intervalSecond'] = intervalSecond
        msg_parts.append('interval: %d' % intervalSecond)
        trigger['periodicSchedule'] = schedule
        
    spec['enabled'] = enabled
    spec['maxHistory'] = maxHistory
    spec['trigger'] = trigger

    if start is not None:
        msg_parts.append('start: %s' % str(start))
    if end is not None:
        msg_parts.append('end: %s' % str(end))

    spec['description'] = 'ScheduledWorkflow (%s): %s' % (', '.join(msg_parts), description)

    swf_name = 'swf-%s' % name
    spec['name'] = swf_name
    metadata = swf['metadata']
    metadata['name'] = swf_name

    workflow = spec['workflow']

    parameters = workflow['parameters']
    parameters[1]['value'] = yaml.dump(pipeline)
    
    workflow_spec = workflow['spec']
    templates = workflow_spec['templates']
    template = templates[0]
    container = template['container']
    args = container['args']
    args[-1] = name

    return swf

In [248]:
swf = make_swf_kfp(condition_zip_path); swf

{'apiVersion': 'kubeflow.org/v1beta1',
 'kind': 'ScheduledWorkflow',
 'metadata': {'name': 'swf-conditional-execution-pipeline'},
 'spec': {'description': 'ScheduledWorkflow (cron: /10 * * * * *): Shows how to use dsl.Condition().',
  'enabled': True,
  'maxHistory': 10,
  'name': 'swf-conditional-execution-pipeline',
  'trigger': {'cronSchedule': {'cron': '/10 * * * * *'}},
  'workflow': {'parameters': [{'name': 'datetime',
     'value': '[[ScheduledTime.20060102-15:04:05]]'},
    {'name': 'pipeline_yaml',
     'value': 'apiVersion: argoproj.io/v1alpha1\nkind: Workflow\nmetadata:\n  annotations: {pipelines.kubeflow.org/pipeline_spec: \'{"description": "Shows how\n      to use dsl.Condition().", "name": "Conditional execution pipeline"}\'}\n  generateName: conditional-execution-pipeline-\nspec:\n  arguments:\n    parameters: []\n  entrypoint: conditional-execution-pipeline\n  serviceAccountName: pipeline-runner\n  templates:\n  - dag:\n      tasks:\n      - arguments:\n          parame

In [244]:
print(yaml.dump(swf))

apiVersion: kubeflow.org/v1beta1
kind: ScheduledWorkflow
metadata: {name: swf-conditional-execution-pipeline}
spec:
  description: 'ScheduledWorkflow (cron: * * * * *): Shows how to use dsl.Condition().'
  enabled: true
  maxHistory: 10
  name: swf-conditional-execution-pipeline
  trigger:
    cronSchedule: {cron: '* * * * *'}
  workflow:
    parameters:
    - {name: datetime, value: '[[ScheduledTime.20060102-15:04:05]]'}
    - {name: pipeline_yaml, value: "apiVersion: argoproj.io/v1alpha1\nkind: Workflow\n\
        metadata:\n  annotations: {pipelines.kubeflow.org/pipeline_spec: '{\"description\"\
        : \"Shows how\n      to use dsl.Condition().\", \"name\": \"Conditional execution\
        \ pipeline\"}'}\n  generateName: conditional-execution-pipeline-\nspec:\n\
        \  arguments:\n    parameters: []\n  entrypoint: conditional-execution-pipeline\n\
        \  serviceAccountName: pipeline-runner\n  templates:\n  - dag:\n      tasks:\n\
        \      - arguments:\n          pa

In [198]:
from kubernetes import client, config
config.load_kube_config()
api = client.CustomObjectsApi()

In [245]:
api.create_namespaced_custom_object(
    group="kubeflow.org",
    version="v1beta1",
    namespace="default",
    plural="scheduledworkflows",
    body=swf,
)

{'apiVersion': 'kubeflow.org/v1beta1',
 'kind': 'ScheduledWorkflow',
 'metadata': {'creationTimestamp': '2019-09-29T02:18:15Z',
  'generation': 1,
  'name': 'swf-conditional-execution-pipeline',
  'namespace': 'default',
  'resourceVersion': '90037',
  'selfLink': '/apis/kubeflow.org/v1beta1/namespaces/default/scheduledworkflows/swf-conditional-execution-pipeline',
  'uid': '6595fbf4-219d-473d-9790-a4cb96036769'},
 'spec': {'description': 'ScheduledWorkflow (cron: * * * * *): Shows how to use dsl.Condition().',
  'enabled': True,
  'maxHistory': 10,
  'name': 'swf-conditional-execution-pipeline',
  'trigger': {'cronSchedule': {'cron': '* * * * *'}},
  'workflow': {'parameters': [{'name': 'datetime',
     'value': '[[ScheduledTime.20060102-15:04:05]]'},
    {'name': 'pipeline_yaml',
     'value': 'apiVersion: argoproj.io/v1alpha1\nkind: Workflow\nmetadata:\n  annotations: {pipelines.kubeflow.org/pipeline_spec: \'{"description": "Shows how\n      to use dsl.Condition().", "name": "Condit

In [246]:
api.delete_namespaced_custom_object(
    name='swf-conditional-execution-pipeline',
    group="kubeflow.org",
    version="v1beta1",
    namespace="default",
    plural="scheduledworkflows",
    body=client.V1DeleteOptions(),
)

{'apiVersion': 'v1',
 'details': {'group': 'kubeflow.org',
  'kind': 'scheduledworkflows',
  'name': 'swf-conditional-execution-pipeline',
  'uid': '6595fbf4-219d-473d-9790-a4cb96036769'},
 'kind': 'Status',
 'metadata': {},
 'status': 'Success'}

In [172]:
pipeline

'/var/folders/m0/mj2x82p1527349z6mn8btgtr0000gr/T/tmp8uhbrus1/condition.py.zip'

In [175]:
with ZipFile(pipeline, 'r') as zip:
    pipeline_yaml = zip.read('pipeline.yaml').decode()

In [177]:
len(pipeline_yaml)

6430

In [178]:
parameters = workflow['parameters']

In [179]:
parameters.append({
    'name': 'pipeline',
    'value': pipeline_yaml
})

In [180]:
len(swf_spec['spec']['workflow']['parameters'])

2

In [176]:
pipeline_yaml

'apiVersion: argoproj.io/v1alpha1\nkind: Workflow\nmetadata:\n  annotations:\n    pipelines.kubeflow.org/pipeline_spec: \'{"description": "Shows how to use dsl.Condition().",\n      "name": "Conditional execution pipeline"}\'\n  generateName: conditional-execution-pipeline-\nspec:\n  arguments:\n    parameters: []\n  entrypoint: conditional-execution-pipeline\n  serviceAccountName: pipeline-runner\n  templates:\n  - dag:\n      tasks:\n      - arguments:\n          parameters:\n          - name: generate-random-number-output\n            value: \'{{tasks.generate-random-number.outputs.parameters.generate-random-number-output}}\'\n        dependencies:\n        - generate-random-number\n        name: condition-2\n        template: condition-2\n        when: \'{{tasks.generate-random-number.outputs.parameters.generate-random-number-output}}\n          > 5\'\n      - arguments:\n          parameters:\n          - name: generate-random-number-output\n            value: \'{{tasks.generate-r

In [171]:
with tarfile.open(pipeline, 'r') as tar:
    pipeline_yaml = tar.extractfile(tar.getmember('pipeline.yaml')).read()

ReadError: file could not be opened successfully

In [166]:
type(swf_spec)

dict

In [130]:
from os.path import basename
import sys
from tempfile import mkdtemp

In [131]:
tmp = mkdtemp()

In [143]:
module_name = basename(module_url)

In [134]:
with open(path(tmp, basename(module_url)), 'w') as f:
    f.write(module_src)

In [136]:
with open(path(tmp, '__init__.py'), 'w') as f:
    f.write('')

In [137]:
sys.path.append(tmp)
from condition import flipcoin_pipeline
sys.path.pop()

In [147]:
pipeline = path(tmp, module_name + '.zip')

In [139]:
from kfp.compiler import Compiler

In [140]:
compiler = Compiler()
compiler.compile(flipcoin_pipeline, pipeline)

In [150]:
from kfp import Client

In [155]:
client = Client()

In [156]:
exp = client.create_experiment('exp'); exp

{'created_at': datetime.datetime(2019, 9, 28, 21, 51, tzinfo=tzlocal()),
 'description': None,
 'id': 'b3777962-676c-41e1-a5e8-47c0fe91830b',
 'name': 'exp'}

In [158]:
from datetime import datetime as dt

In [159]:
client.run_pipeline(exp.id, str(dt.now()), pipeline)

{'created_at': datetime.datetime(2019, 9, 28, 21, 57, 52, tzinfo=tzlocal()),
 'description': None,
 'error': None,
 'finished_at': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
 'id': '074e8e9f-61bd-422d-a2c4-e05fbf9cfb40',
 'metrics': None,
 'name': '2019-09-28 21:57:52.602510',
 'pipeline_spec': {'parameters': None,
                   'pipeline_id': None,
                   'pipeline_manifest': None,
                   'workflow_manifest': '{"apiVersion": '
                                        '"argoproj.io/v1alpha1", "kind": '
                                        '"Workflow", "spec": '
                                        '{"serviceAccountName": '
                                        '"pipeline-runner", "arguments": '
                                        '{"parameters": []}, "entrypoint": '
                                        '"conditional-execution-pipeline", '
                                        '"templates": [{"dag": {"tasks": '
                   

In [160]:
from kubernetes import client, config

# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()

v1 = client.CoreV1Api()

In [161]:
dir(v1)

TypeError: 'str' object is not callable

In [146]:
tmp

'/var/folders/m0/mj2x82p1527349z6mn8btgtr0000gr/T/tmp8uhbrus1'

In [20]:
kfp.__file__

'/Users/ryan/.pyenv/versions/kf-pipelines-py3/lib/python3.5/site-packages/kfp/__init__.py'